<a href="https://colab.research.google.com/github/Kulikov17/MLDS_DL/blob/main/toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>